In [1]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

In [2]:
JAWLINE_POINTS = list(range(0, 17))  
RIGHT_EYEBROW_POINTS = list(range(17, 22))  
LEFT_EYEBROW_POINTS = list(range(22, 27))  
NOSE_POINTS = list(range(27, 36))  
RIGHT_EYE_POINTS = list(range(36, 42))  
LEFT_EYE_POINTS = list(range(42, 48))  
MOUTH_OUTLINE_POINTS = list(range(48, 61))  
MOUTH_INNER_POINTS = list(range(61, 68))

In [3]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
 
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])
 
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
 
	# return the eye aspect ratio
	return ear

In [ ]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.26
EYE_AR_CONSEC_FRAMES = 10
 
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0

TOTAL = 0
ALARM_ON = False

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

#Set up some required objects
video_capture = cv2.VideoCapture(0) #Webcam object
detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #Landmark identifier. Set the filename to whatever you named the downloaded file

while True:
	ret, frame = video_capture.read(0)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	faces = face_cascade.detectMultiScale(gray, 1.1, 5)
	for (x, y, w, h) in faces:  
		cv2.rectangle(frame, (x, y), (x + w, y + w), (0,0, 255), 1) 
		roi_gray = gray[y:y+h, x:x+w]
		roi_color = frame[y:y+h, x:x+w]
   
        
		# Converting the OpenCV rectangle coordinates to Dlib rectangle  
		dlib_rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))  
          #display the face landmarks 
		landmarks = np.matrix([[p.x, p.y] 
				for p in predictor(frame, dlib_rect).parts()])  
		rightEye= landmarks[RIGHT_EYE_POINTS]
		leftEye= landmarks[LEFT_EYE_POINTS]
		landmarks_display = landmarks[RIGHT_EYE_POINTS + LEFT_EYE_POINTS+MOUTH_OUTLINE_POINTS+JAWLINE_POINTS+RIGHT_EYEBROW_POINTS+LEFT_EYEBROW_POINTS+ NOSE_POINTS]
		rightEAR = eye_aspect_ratio(rightEye)
		leftEAR = eye_aspect_ratio(leftEye)

		# average the eye aspect ratio together for both eyes
		ear = (leftEAR + rightEAR) / 2.0
      
		#for idx, point in enumerate(landmarks_display):  
			#pos = (point[0, 0], point[0, 1])  
			#cv2.circle(frame, pos, 2, color=(0, 255, 255), thickness=-1)
		#cv2.imshow("image", frame) #Display the frame
        # check to see if the eye aspect ratio is below the blink
		# threshold, and if so, increment the blink frame counter
		if ear < EYE_AR_THRESH:
			COUNTER += 1
 
		# otherwise, the eye aspect ratio is not below the blink
		# threshold
		else:
			# if the eyes were closed for a sufficient number of
			# then increment the total number of blinks
			if COUNTER >= EYE_AR_CONSEC_FRAMES:
				TOTAL += 1
 
			# reset the eye frame counter
			COUNTER = 0
		#draw the total number of blinks on the frame along with
		# the computed eye aspect ratio for the frame
		cv2.putText(frame, "BlinksCount: {}".format(TOTAL), (10, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		cv2.putText(frame, "EARRatio: {:.2f}".format(ear), (300, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
	# show the frame
		for idx, point in enumerate(landmarks_display):  
			pos = (point[0, 0], point[0, 1])  
			cv2.circle(frame, pos, 2, color=(0, 255, 255), thickness=-1)
		cv2.imshow("Frame", frame)
		key = cv2.waitKey(1) & 0xFF
 
	# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break

# do a bit of cleanup
video_capture.release()
cv2.destroyAllWindows()
video_capture .stop()            